# AE Price Estimation

This notebook introduces the `ae_estimation_price` function, which is part of the *ae_price_estimation* module in the *finance* package of *QQuantLib* (**QQuantLib/finance/ae_price_estimation**).

This function enables users to solve an input **option price estimation** problem using **amplitude estimation** algorithms. 

The `ae_estimation_price` function utilizes the `DensityProbability` and `PayOff` Python classes (explained in notebook **11_ApplicationTo_Finance_02_ClassicalFinance**) to create the corresponding arrays $ p(x) $ and $ f(x) $ defined over a domain $ x \in [x_0, x_f] $. Subsequently, the function attempts to compute the expected value:

$$
\mathbb{E}[f] = \int_{x_0}^{x_f} p(x) f(x) dx
$$

using the `q_solve_integral` function, as explained in notebook **10_ApplicationTo_Finance_01_IntegralComputing**.

Once the `q_solve_integral` function calculates the desired expectation, the `ae_estimation_price` function estimates the option price using:

$$
V(t, S(t)) = e^{-r(T-t)} \int_{x_0}^{x_f} p(x) f(x) dx
$$

In [ ]:
import sys
sys.path.append("../../")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#This cell loads the QLM solver. See notebook: 00_AboutTheNotebooksAndQPUs.ipynb
from QQuantLib.qpu.get_qpu import get_qpu
# myqlm qpus: python, c
# QLM qpus accessed using Qaptiva Access library: qlmass_linalg, qlmass_mps
# QLM qpus: Only in local Quantum Learning Machine: linalg, mps
my_qpus = ["python", "c", "qlmass_linalg", "qlmass_mps", "linalg", "mps"]

linalg_qpu = get_qpu(my_qpus[1])

In [ ]:
from QQuantLib.finance.ae_price_estimation import ae_price_estimation

## 1. ae_price_estimation Input

The input for `ae_price_estimation` is a complete Python dictionary that must include all the necessary keys to define an option price estimation problem and solve it using a properly configured **Amplitude Estimation (AE)** algorithm.

For clarity, this input dictionary can be divided into the following sub-dictionaries:

1. **Domain Configuration**
2. **Probability Density Configuration**
3. **PayOff Configuration**
4. **Encoding Configration**
5. **AE Configuration**
6. **Other Configuration**

### 1.1 Domain Configuration

This dictionary configures the domain and discretization of the problem. In this case, the domain of the problem is defined as:

$$
x \in [x_0, x_f]
$$

For discretization, the domain is split into:

$$
[x_0, x_f] = \{[x_0, x_1] \cup [x_1, x_2] \cup \dots \cup [x_{2^n-1}, x_f]\}
$$

The keys for this *domain* dictionary are:

- `x0`: The starting point of the domain.
- `xf`: The ending point of the domain.
- `n_qbits`: The number of qubits used to divide the domain into $ 2^{n\_qbits} $ intervals.

In [ ]:
#Domain configuration
domain_configuration = {
    'x0': 0.01,
    'xf': 5.0,
    'n_qbits': 5,      
}

### 1.2 Probability Density Configuration

This dictionary contains information related to the underlying asset of the option, market conditions, and the time at which the probability density will be evaluated (typically the *maturity* of the option). This dictionary requires all the necessary keys to properly configure the *DensityProbability* class (from **QQuantLib.finance.probability_class**).

- `probability_type`: A string specifying the type of probability density to use. Currently, only 'Black-Scholes' is available (additional probability density functions are expected to be implemented in the future).
- `s_0`: The initial value of the asset $ S $.
- `risk_free_rate`: The risk-free rate of the market.
- `maturity`: The time at which the probability distribution of the asset's value is computed. When working with options, this corresponds to the *maturity* of the option.
- `volatility`: The volatility of the asset up to the initial time.

In [ ]:
#Probability density configuration
probability_configuration = {
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.5,    
}

### 1.3 PayOff Configuration

This dictionary includes the mandatory keys for configuring the *PayOff* class (from **QQuantLib.finance.payoff_class**). It generally pertains to the type of option, the strike price, and other relevant parameters.

- `pay_off_type`: Specifies the type of option desired. The following options are currently available:
  - European_Call_Option
  - European_Put_Option
  - Digital_Call_Option
  - Digital_Put_Option
  - Futures

- `strike`: The strike price of the derivative product.

- `coupon`: Relevant for *Digital_Call_Option* and *Digital_Put_Option*.

In [ ]:
#PayOff Configuration
payoff_configuration = {
    'pay_off_type': 'European_Call_Option',
    'strike': 0.5,
    'coupon': None,        
}

### 1.4 Encoding Configuration

This dictionary contains all the necessary keys to configure the encoding of the $ p(x) $ and $ f(x) $ arrays into the quantum circuit. These keys are used to properly set up the *Encoding* class (from **QQuantLib.DL.encoding_protocols**) utilized within the `q_solve_integral` function (from **QQuantLib.finance.quantum_integration**).

- `encoding`: The type of encoding used. It can take values 0, 1, or 2.
- `multiplexor`: A flag for using multiplexor constructions for gates controlled by state.

In [ ]:
#For encoding class
encoding_configuration = {
    "encoding" : 0, #1,2
    "multiplexor": True,        
}

### 1.5 AE Configuration

This dictionary contains all the necessary keys for configuring the *Amplitude Estimation (AE)* technique used to solve the integral. These keys are used to properly set up the *q_solve_integral* function (from **QQuantLib.finance.quantum_integration**), which relies on the *AE* class (from **QQuantLib.AE.ae_class**).

The keys for the **AE** configuration will depend on **AE** algorithm selected. See the different related Notebooks for their names.

In [ ]:
m_k = [0, 1, 2, 3, 4, 5, 6, 7, 8]
ae_configuration = {
    #Amplitude Estimation selection
    'ae_type': 'MLAE', #IQAE, RQAE, CQPEAE, IQPEAE    
    
    #MLAE configuration
    'schedule': [
        m_k,
        [100]*len(m_k)
    ],
    'delta' : 1.0e-7,
    'ns' : 10000,
    
    #CQPEAE configuration
    'auxiliar_qbits_number': 14,
    "window" : None,
    "kaiser_alpha" : None,
    
    
    #IQPEAE configuration
    'cbits_number': 10,  
    
    #IQAE & RQAQE
    'epsilon': 0.001,
    #IQAE
    'alpha': 0.05,
    #RQAE
    'gamma': 0.05,
    'q': 1.2,
    #eRQAE
    "erqae_schedule": None,
    
    #shots: applit to IQAE, CQPEAE, IQPEA, sRQAE
    'shots': 100,      
    #Multi controlled decomposition
    'mcz_qlm': False, 
}

### 1.5 Other configuration

This dictionary will be related to other configuration keys like:

- `qpu: type of **QPU** used for simulating the **AE** algorithms
- `save`: boolean for saving or not the results
- `file_name`: string with the complete path for saving results
- `number_of_tests`: for doing several repetitions of the solution of a complete give ae problem

In [ ]:
other_configuration = {
    'qpu': linalg_qpu,
    "save": False,
    "file_name": "./ae_problem.csv",
    "number_of_tests": 1
}

In [ ]:
price_estimation_configuration = {}
price_estimation_configuration.update(domain_configuration)
price_estimation_configuration.update(probability_configuration)
price_estimation_configuration.update(payoff_configuration)
price_estimation_configuration.update(encoding_configuration)
price_estimation_configuration.update(ae_configuration)
price_estimation_configuration.update(other_configuration)

In [ ]:
price_estimation_configuration

## 2. ae_price_estimation Workflow

The `ae_price_estimation` function executes the following workflow:

1. **Domain Creation**: Creates the discretized domain using the keys from the *domain_configuration* section of the input dictionary.

2. **Probability Density Configuration**: Uses the *probability_configuration* section of the input dictionary to create a properly configured **DensityProbability** object.

3. **Probability Distribution Array**: Utilizes the `DensityProbability` object and the domain discretization to generate the NumPy array $ p(x) $ for the probability distribution.

4. **Payoff Configuration**: Uses the *payoff_configuration* section of the input dictionary to create a properly configured `PayOff`object.

5. **Payoff Array**: Utilizes the `PayOff` object and the domain discretization to generate the NumPy array $ f(x) $ for the payoff of the selected derivative option.

6. **Normalization**: Normalizes the $ p(x) $ and $ f(x) $ arrays.

7. **Input Dictionary Update**: Adds the normalized NumPy arrays to the input dictionary.

8. **Quantum Integral Solution**:
    - **Encoding Configuration**: The `q_solve_integral` function uses the *encoding_configuration* section to configure the `Encoding` class appropriately.
    - **AE Configuration**: The `q_solve_integral` function uses the *ae_configuration* section to configure the `AE` class appropriately.
    - **Amplitude Estimation Execution**: The `q_solve_integral` function executes the *AE* algorithm to compute the estimation of the amplitude and the desired integral.

9. **Expected Value Calculation**: The `q_solve_integral` function returns the desired expected value computed using **AE** integral techniques.

10. **Post-Processing**: Processes the results to provide the price estimation solution.

The output of the `ae_price_estimation` function is a pandas DataFrame containing all the configuration information provided, along with the results.


In [ ]:
pdf = ae_price_estimation(**price_estimation_configuration)

In [ ]:
pdf

In [ ]:
pdf.columns

The returned DataFrame has a lot of information. 

For traceability we have include all the keys of the input dictionary:

In [ ]:
#Domain configuration
pdf[['x0', 'xf', 'n_qbits']]

In [ ]:
#Probability configuration
pdf[['probability_type', 's_0', 'risk_free_rate','maturity', 'volatility']]

In [ ]:
#PayOff configuration
pdf[['pay_off_type', 'strike', 'coupon']]

In [ ]:
#ae configuration
pdf[[
    'ae_type','schedule', 'delta', 'ns', 'auxiliar_qbits_number', 'cbits_number',
    'epsilon', 'alpha', 'gamma', 'q', 'shots', 'mcz_qlm', 'encoding','multiplexor', "erqae_schedule"]]

In [ ]:
#other configuration
pdf[['qpu', 'save', 'file_name', 'number_of_tests']]

The normalisation of the numpy arrays $p(x)$ and $f(x)$ are included too:

In [ ]:
pdf[['payoff_normalisation', 'p_x_normalisation']]

The columns related to the **AE** integration are:

- `riemman_expectation`: this is the expectation computed as a Riemann, this is the scalar product of the $p(x)$ and the $f(x)$ arrays. 
- `ae_expectation`: expectation computed using **AE** integration techniques.
- `ae_l_expectation`: lower value of the expectation computed using **AE** integration techniques (for **IQAE** and **RQAE**)
- `ae_u_expectation`: upper value of the expectation computed using **AE** integration techniques (for **IQAE** and **RQAE**)
- `absolute_error`: absolute difference between the **AE** integral estimation and the desired integral (*riemann_expectation*)

In [ ]:
pdf[[
    "ae_l_expectation", "ae_expectation", 
    "ae_u_expectation", "riemann_expectation", "absolute_error"]]

In [ ]:
(np.abs(pdf["ae_expectation"] - pdf["riemann_expectation"]), pdf["absolute_error"])

The columns related with the finance quantities allways beginf by *finance* and are:

- `finance_exact_price`: the price of the option computed using an analytical Black-Scholes model
- `finance_riemann_price`: the price of the option when the discount of the risk free ratio is taken into account when using the Riemman aproximation: $$riemann\_expectation * \; e ^{ r *T}$$ with $r$, the *risk free ratio* and $T$ the *maturity*
- `finance_price_estimation`: the price estimation of the option using **AE**. This is with the discount of the risk free ratio: $$ae\_expectation * \; e ^{ r *T}$$ with $r$, the *risk free ratio* and $T$ the *maturity*
- `finance_error_riemann` : error between the estimated price option and the Rieman obtained price: $$| finance\_price\_estimation -finance\_riemann\_price|$$
- `finance_error_exact`: error between the estimated price option and the exact Rieman obtained price: $$| finance\_price\_estimation - finance\_exact\_price |$$

In [ ]:
pdf[['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact'
]]

In [ ]:
# The different errors that are delivered
pdf[[
    'absolute_error', 'finance_error_riemann', 'finance_error_exact']]

Finally, other important information is provided:

- `schedule_pdf`: this  is the number of Grover-like operator applications and the number of shots for each application (this is valid for **MLAE**, **IQAE** and **RQAE**)
- `oracle_calls`: total number of calls to the oracle for getting the obtained solution.
- `max_oracle_depth`: maximum number of Grover-like operator applications for getting the obtained solution.
- `run_time`:  simulation time for getting the obtained solution.

In [ ]:
pdf[['schedule_pdf']].iloc[0].values

In [ ]:
pdf[['oracle_calls', 'max_oracle_depth', 'run_time']]

## 3. Considerations

Several considerations about the *ae_price_estimation* function are summarised in this section.

### 3.1 Encoding considerations.

As explained in notebook *09_DataEncodingClass.ipynb* there are 3 different encodings: 0, 1 y 2. The encoding 0 can be used for encoding strictly defined positive functions $f(x)$. If this condition is not satisfied then encodings 1 and 2 should be used. For pure price estimation, the payoffs will satisfy this condition so encoding 0 is enough. In our case, encoding 0 can be used for:

* European_Call_Option
* European_Put_Option
* Digital_Call_Option
* Digital_Put_Option

In the case of the *Futures* the function can not satisfy the positive defined function $f(x)$ so for this derivative product encodings 1 and 2 should be used!!

In [ ]:
#Future with encoding 0
price_estimation_configuration.update({
    'pay_off_type': 'Futures',
    'strike': 0.8,
    'encoding': 0,
    'ae_type': 'IQAE'
})

In [ ]:
pdf_future = ae_price_estimation(**price_estimation_configuration)

In [ ]:
#Amplitude Estimation Integral computation results
pdf_future[[
    "ae_l_expectation", "ae_expectation", 
    "ae_u_expectation", "riemann_expectation", "absolute_error"]]

In [ ]:
# Finance related prices
pdf_future[['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact'
]]

As can be seen in the before cells the solution for our future using encoding 0 is very bad. Encoding 1 or 2 should be used instead!!

In [ ]:
#Future with encoding 2
price_estimation_configuration.update({
    'pay_off_type': 'Futures',
    'strike': 0.8,
    'encoding': 2,
    'ae_type': 'IQAE'
})

In [ ]:
pdf_future = ae_price_estimation(**price_estimation_configuration)

In [ ]:
#Amplitude Estimation Integral computation results
pdf_future[[
    "ae_l_expectation", "ae_expectation", 
    "ae_u_expectation", "riemann_expectation", "absolute_error"]]

In [ ]:
# Finance related prices
pdf_future[['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact'
]]

With the encoding 2 the solution of the Future is now well estimated!!!

### 3.2 Encoding 0 and RQAE methods.

When using the encoding 0 the **RQAE** technique (and its variants like **eRQAE**, **mRQAE** and **sRQAE**) can not be used. Instead of an error a warning is raised. The *ae_price_estimation* function provides output pandas DataFrame but the columns related with the results will have a None.

In [ ]:
#Future with encoding 0
price_estimation_configuration.update({
    'pay_off_type': 'Futures',
    'strike': 0.8,
    'encoding': 0,
    'ae_type': 'RQAE'#eRQAE, sRQAE, mRQAE
})

In [ ]:
pdf_future = ae_price_estimation(**price_estimation_configuration)

In [ ]:
pdf_future

In [ ]:
#Amplitude Estimation Integral computation results
pdf_future[[
    "ae_l_expectation", "ae_expectation", 
    "ae_u_expectation", "riemann_expectation", "absolute_error"]]

In [ ]:
# Finance related prices
pdf_future[['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact'
]]

### 3.3 Negative integrals and RQAE

For pure option price estimation (European_Call_Option, European_Put_Option, Digital_Call_Option, Digital_Put_Option) the price always will be positive so all the **AE** methods can be used (of course the 3 encoding methods can be used). However, there are other financial products where the expected return is needed and it is not necessarily positive (for example de Futures). In these cases, the use of encoding 1 or 2 is not enough for getting the correct results. If the solution is negative the only **AE** technique suitable will be the **RQAE** or their variants (**sRQAE**, **mRQAE** or **eRQAE**).

In [ ]:
#Future with encoding 2
price_estimation_configuration.update({
    'pay_off_type': 'Futures',
    'strike': 1.5,
    'encoding': 2,
    'ae_type': 'IQAE'
})

In [ ]:
pdf_future = ae_price_estimation(**price_estimation_configuration)

In [ ]:
#Amplitude Estimation Integral computation results
pdf_future[[
    "ae_l_expectation", "ae_expectation", 
    "ae_u_expectation", "riemann_expectation", "absolute_error"]]

In [ ]:
# Finance related prices
pdf_future[['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact'
]]

As can be seen when the expectation is negative the **AE** method fails despite of the encoding procedure

In [ ]:
#Future with encoding 2
price_estimation_configuration.update({
    'pay_off_type': 'Futures',
    'strike': 1.5,
    'encoding': 2,
    'ae_type': 'RQAE'
})
pdf_future = ae_price_estimation(**price_estimation_configuration)

In [ ]:
#Amplitude Estimation Integral computation results
pdf_future[[
    "ae_l_expectation", "ae_expectation", 
    "ae_u_expectation", "riemann_expectation", "absolute_error"]]

In [ ]:
# Finance related prices
pdf_future[['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact'
]]

As can be seen the **RQAE** algorithm give us the correct value!!!

#### mRQAE

In [ ]:
#Future with encoding 2
price_estimation_configuration.update({
    'pay_off_type': 'Futures',
    'strike': 1.5,
    'encoding': 2,
    'ae_type': 'mRQAE'
})
pdf_future_mrqae = ae_price_estimation(**price_estimation_configuration)
#Amplitude Estimation Integral computation results
print(pdf_future_mrqae[[
    "ae_l_expectation", "ae_expectation", 
    "ae_u_expectation", "riemann_expectation", "absolute_error"]]
)
print(# Finance related prices
pdf_future_mrqae[['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact'
]])

#### sRQAE

In [ ]:
#Future with encoding 2
price_estimation_configuration.update({
    'pay_off_type': 'Futures',
    'strike': 1.5,
    'encoding': 2,
    'ae_type': 'sRQAE',
    'shots': 378
})
pdf_future_srqae = ae_price_estimation(**price_estimation_configuration)
#Amplitude Estimation Integral computation results
print(pdf_future_srqae[[
    "ae_l_expectation", "ae_expectation", 
    "ae_u_expectation", "riemann_expectation", "absolute_error"]]
)
print(# Finance related prices
pdf_future_srqae[['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact'
]])

In [ ]:
pdf_future_srqae["shots"]

#### eRQAE

In [ ]:
#Future with encoding 2

price_estimation_configuration.update({
    'pay_off_type': 'Futures',
    'strike': 1.5,
    'encoding': 2,
    'ae_type': 'eRQAE',
    'erqae_schedule': {
        "type": "linear_linear",
        "ratio_slope_k": 2, 
        "ratio_slope_gamma": 2
    }
})
pdf_future_erqae = ae_price_estimation(**price_estimation_configuration)
#Amplitude Estimation Integral computation results
print(pdf_future_erqae[[
    "ae_l_expectation", "ae_expectation", 
    "ae_u_expectation", "riemann_expectation", "absolute_error"]]
)


In [ ]:
pdf_future_erqae[['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact'
]]

In [ ]:
pdf_future_erqae["erqae_schedule"]

### 3.4 CQPEAE and IQPEAE  techniques

The *ae_price_estimation* can be used with all the encodings and all the **AE** algorithms implemented in the **QQuantLib**. However, a piece of advice should be given when **CQPEAE** and **IQPEAE** algorithms are used. These algorithms used pure **Quantum Phase Estimation** for computing the integrals. These kinds of methods are very computing demand and usually large simulation times are needed for solving the price estimation problem. 

## 4. Loading PayOffs by Parts

As explained in this and other notebooks, a key challenge when using **AE** estimation for computing integrals (or expected values) arises when the function to be integrated is negative over some part of the domain. While Encoding 1 and 2 can address this issue, most **AE** algorithms (except **RQAE**) fail if the final integral is negative, as they return the probability rather than the amplitude.

One solution to this problem is to separately load the positive and negative parts of the function, execute the **AE** estimation for each part, and then post-process the results to obtain the final result. The workflow is as follows:

1. Load the positive part of $ f(x) $ and use the **AE** algorithm to obtain the estimation $ a^+ $ (and its corresponding semi-difference bounded interval: $ \epsilon_a^+ $, if provided by the **AE** algorithm).
2. Load the negative part of $ f(x) $ (in this case, load the absolute value of the function: $ |f(x)| $) and use the **AE** algorithm to obtain the estimation $ a^- $ (and its corresponding semi-difference bounded interval: $ \epsilon_a^- $, if provided by the **AE** algorithm). Note that $ a^- > 0 $.
3. Post-process the obtained results to calculate the final estimation: $ a_{final} = a^+ - a^- $.
4. The associated semi-difference bound interval will be the sum of the obtained intervals: $ \epsilon_a = \epsilon_a^+ + \epsilon_a^- $.

With this procedure, any **AE** algorithm can be used to estimate the desired integral. Additionally, this approach is compatible with all encodings: 0 (or square encoding), 1, and 2 (or direct encoding).

The `ae_price_estimation_step_po` function from the **QQuantLib.finance.ae_price_estimation_step_payoff** module implements this procedure for a complete price estimation problem and **AE** algorithm. This function operates similarly to the *ae_price_estimation* function described in this notebook.

The `ae_price_estimation_step_po` function returns a pandas DataFrame with the same configuration as that returned by the **ae_price_estimation** function, but it includes additional columns for the negative and positive estimation parts:
- `ae_positive_part`: Estimation of the positive part.
- `ae_l_positive_part`: Lower bound for the positive part estimation.
- `ae_u_positive_part`: Upper bound for the positive part estimation.
- `ae_negative_part`: Estimation of the negative part.
- `ae_l_negative_part`: Lower bound for the negative part estimation.
- `ae_u_negative_part`: Upper bound for the negative part estimation.

In [ ]:
from QQuantLib.finance.ae_price_estimation_step_payoff import ae_price_estimation_step_po

In [ ]:
future_pe_conf = {
    # Domain
    'x0': 0.01,
    'xf': 5.0,
    'n_qbits': 5,       
    # Probability
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.5,    
    # PayOff
    'pay_off_type': 'Futures',
    'strike': 1.5,
    #AE Configuration
    #MLAE configuration
    'schedule': [
        m_k,
        [100]*len(m_k)
    ],
    'delta' : 1.0e-7,
    'ns' : 10000,
    
    #CQPEAE configuration
    'auxiliar_qbits_number': 14,
    
    #IQPEAE configuration
    'cbits_number': 10,  
    
    #IQAE & RQAQE
    'epsilon': 0.0001,
    #IQAE
    'alpha': 0.05,
    #RQAE
    'gamma': 0.05,
    'q': 2.0,
    #eRQAE
    "erqae_schedule": None,
    
    #shots: applit to IQAE, CQPEAE, IQPEA, sRQAE
    'shots': 100,      
    #Multi controlled decomposition
    'mcz_qlm': False,    
}
future_pe_conf.update({"qpu": linalg_qpu})

In [ ]:
# We are going to use the square encoding
future_pe_conf.update({"encoding": 0})
future_pe_conf.update({"ae_type": "IQAE"})

step_future_square = ae_price_estimation_step_po(
    **future_pe_conf)

In [ ]:
# positive part estimation
step_future_square[["ae_positive_part", "ae_l_positive_part", "ae_u_positive_part"]]

In [ ]:
# negative part estimation
step_future_square[["ae_negative_part", "ae_l_negative_part", "ae_u_negative_part"]]

Now the estimation is correct:

In [ ]:
step_future_square[
    ["ae_expectation", "measured_epsilon", "riemann_expectation", "absolute_error"]
]

In [ ]:
step_future_square[
    ['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact']
]

In [ ]:
# We are going to use the direct encoding
future_pe_conf.update({"encoding": 2})
future_pe_conf.update({"ae_type": "IQAE"})

step_future_square = ae_price_estimation_step_po(
    **future_pe_conf)

Again the estimation is correct

In [ ]:
step_future_square[
    ["ae_expectation", "measured_epsilon", "riemann_expectation", "absolute_error"]
]

In [ ]:
step_future_square[
    ['finance_exact_price', 'finance_riemann_price', 
     'finance_price_estimation', 'finance_error_riemann', 'finance_error_exact']
]

## 5. Amplitude Estimation Algorithms comparison

The **benchmark.q_ae_price.benchmark_ae_option_price.py** script allows to the user generate, in an easy way, combinations of different price estimation problems and **AE** algorithm configurations for feeding to the **ae_price_estimation** function of the **QQuantLib.finance.ae_price_estimation** module.


The **benchmark.q_ae_price.benchmark_ae_option_price_step_po.py** script allows to the user generate, in an easy way, combinations of different price estimation problems and **AE** algorithm configurations for feeding to the **ae_price_estimation_step_po** of the **QQuantLib.finance.ae_price_estimation_step_payoff** module.

These two scripts can be used for generating benchmark simulations automatically. We refer to the notebook  *Compare_AE_algorithms_On_PriceEstimation.ipynb* (in **benchmark.q_ae_price** package) for more info.